In [1]:
from compas.geometry import Point, Vector, Frame, Box, close

from scipy.spatial import cKDTree

boxes = [Box(4.5, 9.5, 3.0, Frame(Point(2.250, 4.750, 1.500), Vector(1.000, 0.000, 0.000), Vector(0.000, 1.000, 0.000))),
#Box(3.0, 5.0, 3.0, Frame(Point(1.500, 2.500, 1.500), Vector(1.000, 0.000, 0.000), Vector(0.000, 1.000, 0.000))),
]
#Box(8.0, 4.5, 3.0, Frame(Point(4.000, 2.250, 1.500), Vector(1.000, 0.000, 0.000), Vector(0.000, 1.000, 0.000))),
#Box(5.0, 5.0, 3.0, Frame(Point(2.500, 2.500, 1.500), Vector(1.000, 0.000, 0.000), Vector(0.000, 1.000, 0.000))),
#Box(7.5, 6.5, 3.0, Frame(Point(3.750, 3.250, 4.50), Vector(1.000, 0.000, 0.000), Vector(0.000, 1.000, 0.000))),
#Box(5.0, 6.5, 3.0, Frame(Point(2.500, 3.250, 4.50), Vector(1.000, 0.000, 0.000), Vector(0.000, 1.000, 0.000)))]

tt_faces = [[Point(5.000, 8.000, 6.000), Point(0.000, 8.000, 6.000), Point(5.000, 3.000, 6.000), Point(0.000, 3.000, 6.000)],
[Point(14.500, -1.500, 6.000), Point(14.500, -1.500, 3.000), Point(12.500, -1.500, 6.000), Point(12.500, -1.500, 3.000)],
[Point(12.500, -1.500, 6.000), Point(12.500, 8.000, 6.000), Point(14.500, 8.000, 6.000), Point(14.500, -1.500, 6.000), Point(12.500, -1.500, 6.000)],
[Point(12.500, -1.500, 3.000), Point(12.500, 8.000, 3.000), Point(14.500, 8.000, 3.000), Point(14.500, -1.500, 3.000), Point(12.500, -1.500, 3.000)],
[Point(14.500, 8.000, 4.200), Point(14.500, 8.000, 3.000), Point(14.500, -1.500, 4.200), Point(14.500, -1.500, 3.000)],
[Point(12.500, 8.000, 6.000), Point(12.500, 8.000, 3.000), Point(14.500, 8.000, 6.000), Point(14.500, 8.000, 3.000)],
]

points = []
for box in boxes:
    for pt in box.to_vertices_and_faces()[0]:
        if not len(points):
            points.append(pt)
            continue

        tree = cKDTree(points)
        d, idx = tree.query(pt, k=1)
        if close(d, 0, 1e-3):
            pass
        else:
            points.append(pt)

for face in tt_faces:
    for pt in face:
        tree = cKDTree(points)
        d, idx = tree.query(pt, k=1)
        if close(d, 0, 1e-3):
            pass
        else:
            points.append(pt)

print(len(points))
tree = cKDTree(points)

from compas.datastructures import CellNetwork

network = CellNetwork()

for i, (x, y, z) in enumerate(points):
    network.add_vertex(key=i, x=x, y=y, z=z)

for box in boxes:
    verts, faces = box.to_vertices_and_faces()
    fidxs = []
    for face in faces:
        nface = []
        for idx in face:
            pt = verts[idx]
            d, ni = tree.query(pt, k=1)
            nface.append(ni)
        fidx = None
        for face in network.faces():
            if set( network.face_vertices(face)) == set(nface):
                fidx = face
                break
        else:
            fidx = network.add_face(nface)
        fidxs.append(fidx)
    network.add_cell(fidxs)
    print(fidxs)


for face in tt_faces:

    nface = []
    for pt in face:
        d, ni = tree.query(pt, k=1)
        nface.append(ni)

    fidx = None
    for face in network.faces():
        if set( network.face_vertices(face)) == set(nface):
            fidx = face
            break
    else:
        fidx = network.add_face(nface)




"""
 [network.add_vertex(x=x, y=y, z=z) for x, y, z in vertices]
    [network.add_edge(u, v) for u, v in edges]
    [network.add_face(fverts) for fverts in faces]
    [network.add_cell(fkeys) for fkeys in cells]
"""
network.to_json('/Users/romanarust/workspace/compas_dev/compas/docs/userguide/network.json')


22
[0, 1, 2, 3, 4, 5]


In [2]:
from compas.geometry import Box
from compas.datastructures import CellNetwork

network = CellNetwork()

vertices, faces = Box(1).to_vertices_and_faces()
for x, y, z in vertices:
    network.add_vertex(x=x, y=y, z=z)
print(faces)
fkeys = []
for face in faces:
    fkeys.append(network.add_face(face))
network.add_cell(fkeys)


print(network)

    


[[0, 1, 2, 3], [0, 3, 5, 4], [3, 2, 6, 5], [2, 1, 7, 6], [1, 0, 4, 7], [4, 5, 6, 7]]
<CellNetwork with 8 vertices, 6 faces, 1 cells, 12 edges>


In [7]:
from compas.datastructures import CellNetwork
cell_network = CellNetwork()
vertices = [(0, 0, 0), (0, 1, 0), (1, 1, 0), (1, 0, 0), (0, 0, 1), (1, 0, 1), (1, 1, 1), (0, 1, 1)]

print(len(vertices))
faces = [[0, 1, 2, 3], [0, 3, 5, 4],[3, 2, 6, 5], [2, 1, 7, 6],[1, 0, 4, 7],[4, 5, 6, 7]]
cells = [[0, 1, 2, 3, 4, 5]]
[network.add_vertex(x=x, y=y, z=z) for x, y, z in vertices]
[cell_network.add_face(fverts) for fverts in faces]
[cell_network.add_cell(fkeys) for fkeys in cells]
cell_network

8


KeyError: 0

In [2]:
from compas_notebook.viewer import Viewer
viewer = Viewer()
#viewer.scene.add(mesh, color='#cccccc')

cell_network = network
from compas.geometry import Polygon, Polyhedron

opacity = 0.5


""" 
for face in cell_network.faces_on_boundaries():
    vertices = cell_network.face_coordinates(face)
    viewer.scene.add(Polygon(vertices), color='#cccccc')


for face in cell_network.faces_without_cell():
    vertices = cell_network.face_coordinates(face)
    viewer.scene.add(Polygon(vertices), color="#7d7eff")
"""

for face in cell_network.faces():

    vertices = cell_network.face_coordinates(face)
    try:
        viewer.scene.add(Polygon(vertices), color="#7d7eff")
    except:
        print(face, vertices)


"""
for face in cell_network.faces():
    cells = cell_network.face_cells(face)
    vertices = cell_network.face_coordinates(face)

    if not len(cells):
        pass
        # viewer.add(Polygon(vertices), facecolor=[0.5, 0.55, 1.0], opacity=opacity)

    elif len(cells) == 2:
        pass
        # viewer.add(Polygon(vertices), facecolor=[1, 0.8, 0.05], opacity=opacity)
    # break
"""




viewer.show()

PyThreeJS SceneObjects registered.
No triangles found
No triangles found
No triangles found
No triangles found
